In [ ]:
'''
import your life
'''
import numpy as np
import io
import datetime as date
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def replace_outliers_with_nan(df, column,no0=True,bounds=True,lowerbound=''):
  """
  Edits a dataframe terns outliers (useing the inerqotil rang) and zeros in to NAs. If it is unwanted to thern zeros into na no0=False.

  Returns:
  - df edited.
  """
  df[column] = pd.to_numeric(df[column], errors='coerce') #makeing everthing pd readable
  if no0: #If we are wating to drop the 0'z
      df.loc[df[column] == 0, column] = np.nan
    # Calculate quantiles and IQR
  df[column] = pd.to_numeric(df[column], errors='coerce')
  q1 = np.nanquantile(df[column], 0.25)
  q3 = np.nanquantile(df[column], 0.75)
  iqr = q3 - q1
    # Calculate upper and lower bounds
  upper_bound = q3 + (1.5 * iqr)
  lower_bound = q1 - (1.5 * iqr)
  if bounds: #If we are wanting to control wher the lower bound is (needs to be retooled for upper_bound)
      # Replace outliers with NaN
    df.loc[df[column] < lower_bound, column] = np.nan
    df.loc[df[column] > upper_bound, column] = np.nan
  else:
    df.loc[df[column] < lowerbound, column] = np.nan
    df.loc[df[column] > upper_bound, column] = np.nan
  return df


In [ ]:
Main = pd.read_excel('data-SixForksOnrampMainlineVol-Apr25-May29.xlsx') #reads in main line data
print(Main)

In [ ]:
On14=pd.read_csv('data-SixForksOnrampPassageVolume-May14-29.csv') #reads in Onramp data

In [ ]:
On14=On14.drop('speed',axis='columns')
On14


In [ ]:
On14['volume']=On14['volume']*12 #this is to akont for the when the data is being aerated and the time period looks to be 15 minutes

In [ ]:
On25=pd.read_csv('data-Six Forks Onramp Passage Volume-Apr25toMay14.csv')
On25.head(3726)
On25_1 = On25.iloc[:3724]  #seprats at the different time agerashons
On25_2 = On25.iloc[3724:]
#3724
On25_1['volume'] =On25_1['volume'] *12 #time period looks to be 5 minutes
On25_2["volume"] = On25_2["volume"] * 2 #time period looks to be 30 minutes

In [ ]:
On22=pd.read_csv('Six Forks Onramp Passage Volume 5-min-5-29to7-15.csv')
On22['volume'] = On22['volume']*12
On22=On22.drop('speed',axis='columns')

In [ ]:
On=pd.concat([On14, On25_1,On25_2,On22], ignore_index=True, axis=0,join="inner") #conbining the 4 time periods
On

In [ ]:
Occ25=pd.read_csv('data-Six Forks Onramp Queue Occ-Apr25toMay14.csv') #reads in the Queue occupancy
Occ14=pd.read_csv('data-SixForksOnrampQueueOcc-May14-29.csv')
Occ22=pd.read_csv('Six Forks Onramp Queue Occ 1-min-5-29to7-16.csv')

In [ ]:
Occ=pd.concat([Occ14,Occ25,Occ22],ignore_index=True,axis=0,join="inner")
Occ=Occ.drop(['volume','speed'],axis=1)

In [ ]:
Occ['timestamp'] = pd.to_datetime(Occ['timestamp']) #converting to dt
Occ['timestamp_local'] = Occ['timestamp'].dt.tz_convert('US/Eastern').dt.tz_localize(None) #locilsing
On['timestamp'] = pd.to_datetime(On['timestamp'])
On['timestamp_local'] = On['timestamp'].dt.tz_convert('US/Eastern').dt.tz_localize(None)
Occ = Occ.sort_values(by='timestamp_local')
Occ

In [ ]:
Occ['detector_group_id']="Onramp Queue"
On['detector_group_id']="Onramp Passage"
Occ.rename(columns = {'occupancy':'occupancy Occ','detector_group_id':'detector_group'}, inplace = True) #renaming everthing for the joins
On.rename(columns={'occupancy':'occupancy On','volume':'volume On','detector_group_id':'detector_group'}, inplace = True) #renaming everthing for the joins
Occ=Occ.drop(['timestamp'],axis=1)
On=On.drop(['timestamp'],axis=1)
#Occ 49056
#On #8318
#df 3287

In [ ]:
Main

In [ ]:
#adding the new days
Main22=pd.read_csv('AGG-Six Forks Mainline VOS 15-min-5-29to7-24.csv')
Main22

In [ ]:
Main22['timestamp'] = pd.to_datetime(Main22['timestamp'])
Main22['timestamp'] = Main22['timestamp'].dt.tz_convert('US/Eastern').dt.tz_localize(None)
Main22.rename(columns = {'occupancy':'occupancy (pct)','volume':'volume (vph)','speed':'speed (mph)','timestamp':'timestamp_local'}, inplace = True)
Main22=Main22.drop(['detector_group_id'],axis=1) #wich decter am I filtering for or should use all of them
Main22['volume (vph)'] = Main22['volume (vph)']*4

In [ ]:
Main22

In [ ]:
Main=pd.concat([Main,Main22],ignore_index=True,axis=0,join="inner")
Main= Main.sort_values(by='timestamp_local')
Main

In [ ]:
import plotly.express as px
fig = px.line(Main, x='timestamp_local', y='volume (vph)')
fig.show()
#volume (vph){5/29 1:45-6/6 21:30}
#occupancy (pct){5/29 2:45-6/6 21:30}
#speed (mph){,5/29 1:45-6/6 21:30}

In [ ]:
temp = pd.merge(left=On,right=Occ,how="left",on='timestamp_local').fillna("") #mering 1
df = pd.merge(left=temp,right=Main,how="left",on='timestamp_local').fillna("") #merging 2
df

In [ ]:
import plotly.graph_objects as go #cheking for down times
fig = go.Figure()
#What is with the weredd gap in main?
fig.add_trace(go.Scatter(x=df['timestamp_local'], y=df['volume On'], mode='lines', name='volume On'))
fig.add_trace(go.Scatter(x=df['timestamp_local'], y=df['occupancy On'], mode='lines', name='occupancy On'))
fig.add_trace(go.Scatter(x=df['timestamp_local'], y=df['occupancy Occ'], mode='lines', name='occupancy Occ'))
#'volume On'{4/25 23:00-4/25 10:30,5/29 1:30-6/6 21:35}
#occupancy On'{4/25 23:00-4/25 10:30, 5/29 1:30-6/6 21:35}#with some data errors of greater then 100
#occupancy Occ{5/29 1:30-6/6 21:35}#with some data errors of greater then 100
fig.show()

In [ ]:
df['weekday'] = df['timestamp_local'].dt.weekday #adds a weekdate column where 0 Monday and Sunday is 6

In [ ]:
AMPeak={}
for index, row in df.iterrows():
  if df['weekday'].iloc[index] < 5:
    if df['timestamp_local'].iloc[index].hour >= 7 and df['timestamp_local'].iloc[index].hour < 9:
      AMPeak[index] = True
    else:
        AMPeak[index] = np.nan
  else:
        AMPeak[index] = np.nan
df['AMPeak']= pd.Series(AMPeak) #adds col based if it is a weekday and if it is between the hors of 7AM to 9AM

In [ ]:
df['time'] = df['timestamp_local'].dt.strftime('%H:%M') #Adding a new column Time

In [ ]:
dfs = {}
for time, group in df.groupby('time'): #groups by time and then makes separate dataframe in a dictionary for each time
    group = group.drop(columns=['time'])
    dfs[time] = group

In [ ]:
for time in dfs:
  dfs[time] = replace_outliers_with_nan(dfs[time], 'volume On') #replaceing all outliers with na
  dfs[time] = replace_outliers_with_nan(dfs[time], 'occupancy On',no0=False,bounds=False,lowerbound=0)
  dfs[time] = replace_outliers_with_nan(dfs[time], 'occupancy Occ',no0=False,bounds=False,lowerbound=0)
  dfs[time] = replace_outliers_with_nan(dfs[time], 'volume (vph)')
  dfs[time] = replace_outliers_with_nan(dfs[time], 'occupancy (pct)', no0=False,bounds=False,lowerbound=0)
  dfs[time] = replace_outliers_with_nan(dfs[time], 'speed (mph)')


In [ ]:
df2 = pd.concat(dfs.values())

In [ ]:
LOS = {}
for c, row in df2.iterrows(): #crating bins for speed and other basted on predfid values
    this = df2['speed (mph)'].loc[c]

    if not pd.isnull(this):  # Check for NaN values
        try:
            this = float(this)
            if this == 0:
                LOS[c] = 0
            elif this <= 20:
                LOS[c] = "20 or less"
            elif this <= 30:
                LOS[c] = "20's"
            elif this <= 40:
                LOS[c] = "30's"
            elif this <= 50:
                LOS[c] = "40's"
            elif this <= 60:
                LOS[c] = "50's"
            elif this <= 70:
                LOS[c] = "60's"
            elif this <= 80:
                LOS[c] = "70's"
            elif this <= 90:
                LOS[c] = "80's"
            else:
                LOS[c] = "Greater than 90"
        except ValueError:
            LOS[c] = '' # Handle cases where conversion to float fails
    else:
        LOS[c] = ''  # Handle NaN values

df2['Speedgroup'] = pd.Series(LOS)

In [ ]:
LOS = {}
for c, row in df2.iterrows():
    this = df2['volume (vph)'].loc[c]

    if not pd.isnull(this):  # Check for NaN values
        try:
            this = float(this)
            if this == 0:
                LOS[c] = 0
            elif this <= 1000:
                LOS[c] = "1000"
            elif this <= 1500:
                LOS[c]= "1500"
            elif this <= 2000:
                LOS[c] = "2000"
            elif this <= 2500:
                LOS[c] = "2500"
            elif this <= 3000:
                LOS[c] = "3000"
            elif this <= 3500:
                LOS[c] = "3500"
            elif this <= 4000:
                LOS[c] = "4000"
            else:
                LOS[c] = "Greater than 4000"
        except ValueError:
            LOS[c] = '' # Handle cases where conversion to float fails
    else:
        LOS[c] = ''  # Handle NaN values

df2['Volume bins'] = pd.Series(LOS)

In [ ]:
LOS = {}
for c, row in df2.iterrows():
    this = df2['volume On'].loc[c]

    if not pd.isnull(this):  # Check for NaN values
        try:
            this = float(this)
            if this == 0:
                LOS[c] = 0
            elif this <= 500:
                LOS[c] = "500"
            elif this <= 750:
                LOS[c] = "750"
            elif this <= 1000:
                LOS[c] = "1000"
            elif this <= 1250:
                LOS[c]= "1250"
            elif this <= 1500:
                LOS[c] = "1500"
            elif this <= 1750:
                LOS[c] = "1750"
            elif this <= 2000:
                LOS[c] = "2000"
            elif this <= 2500:
                LOS[c] = "2500"
            elif this <= 3000:
                LOS[c] = "3000"
            elif this <= 3500:
                LOS[c] = "3500"
            elif this <= 4000:
                LOS[c] = "4000"
            else:
                LOS[c] = "Greater than 4000"
        except ValueError:
            LOS[c] = '' # Handle cases where conversion to float fails
    else:
        LOS[c] = ''  # Handle NaN values

df2['Volume Bins On'] = pd.Series(LOS)

In [ ]:
LOS = {}
for c, row in df2.iterrows():
    this = df2['occupancy On'].loc[c]

    if not pd.isnull(this):  # Check for NaN values
        try:
            this = float(this)
            if this == 0:
                LOS[c] = 0
            elif this <= 10:
                LOS[c] = "10%"
            elif this <= 20:
                LOS[c] = "20%"
            elif this <= 30:
                LOS[c] = "30%"
            elif this <= 40:
                LOS[c] = "40%"
            elif this <= 50:
                LOS[c] = "50%"
            elif this <= 60:
                LOS[c] = "60%"
            elif this <= 70:
                LOS[c] = "70%"
            elif this <= 80:
                LOS[c] = "80%"
            elif this <= 90:
                LOS[c] = "90%"
            else:
                LOS[c] = "100%"
        except ValueError:
            LOS[c] = '' # Handle cases where conversion to float fails
    else:
        LOS[c] = ''  # Handle NaN values

df2['occupancy On Bins'] = pd.Series(LOS)

In [ ]:
LOS = {}
for c, row in df2.iterrows():
    this = df2['occupancy (pct)'].loc[c]

    if not pd.isnull(this):  # Check for NaN values
        try:
            this = float(this)
            if this == 0:
                LOS[c] = 0
            elif this <= 10:
                LOS[c] = "10%"
            elif this <= 20:
                LOS[c] = "20%"
            elif this <= 30:
                LOS[c] = "30%"
            elif this <= 40:
                LOS[c] = "40%"
            elif this <= 50:
                LOS[c] = "50%"
            elif this <= 60:
                LOS[c] = "60%"
            elif this <= 70:
                LOS[c] = "70%"
            elif this <= 80:
                LOS[c] = "80%"
            elif this <= 90:
                LOS[c] = "90%"
            else:
                LOS[c] = "100%"
        except ValueError:
            LOS[c] = '' # Handle cases where conversion to float fails
    else:
        LOS[c] = ''  # Handle NaN values

df2['occupancy Bins'] = pd.Series(LOS)

In [ ]:
df2

In [ ]:
df3=df2.drop(['detector_group_x','detector_group_y'],axis='columns') #drops the detector group

In [ ]:
df3.to_csv('SixForks Mainline.csv', sep=',') #Write as csv